In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

from modules.data import *
from modules.stockscreener import *
from modules.stock import Stock
from modules.model import *
from modules.pyportfolioopt import weights_max_sharpe

2023-09-29 12:35:57.985683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Overview

In [3]:
all_stocks = get_data()
all_stocks

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,Topics
0,A,Agilent Technologies Inc. Common Stock,$113.99,-2.2700,-1.953%,3.335202e+10,United States,1999.0,1789767,Industrials,Biotechnology: Laboratory Analytical Instruments,manufacturing
1,AA,Alcoa Corporation Common Stock,$28.10,-0.3500,-1.23%,5.014434e+09,United States,2016.0,4520877,Industrials,Aluminum,manufacturing
2,AAIC,Arlington Asset Investment Corp Class A (new),$4.72,0.1600,3.509%,1.381995e+08,United States,NaN,98323,Real Estate,Real Estate Investment Trusts,real_estate
3,AAIN,Arlington Asset Investment Corp 6.000% Senior ...,$23.50,-0.0900,-0.382%,6.880698e+08,United States,NaN,2009,Real Estate,Real Estate Investment Trusts,real_estate
4,AAP,Advance Auto Parts Inc.,$64.61,1.2200,1.925%,3.842374e+09,United States,NaN,1755190,Consumer Discretionary,Auto & Home Supply Stores,retail_wholesale
...,...,...,...,...,...,...,...,...,...,...,...,...
5146,ZVRA,Zevra Therapeutics Inc. Common Stock,$5.26,-0.0100,-0.19%,1.784857e+08,United States,NaN,76069,Health Care,Biotechnology: Pharmaceutical Preparations,life_sciences
5147,ZVSA,ZyVersa Therapeutics Inc. Common Stock,$0.126,0.0031,2.522%,3.892668e+06,United States,2022.0,39324734,Health Care,Biotechnology: Pharmaceutical Preparations,life_sciences
5148,ZYME,Zymeworks Inc. Common Stock,$7.12,0.1600,2.299%,4.829260e+08,United States,NaN,368909,Health Care,Biotechnology: Pharmaceutical Preparations,life_sciences
5149,ZYNE,Zynerba Pharmaceuticals Inc. Common Stock,$1.32,0.0000,0.00%,7.120005e+07,United States,2015.0,486945,Health Care,Biotechnology: Pharmaceutical Preparations,life_sciences


In [4]:
AAPL = Stock('AAPL')
AAPL.get_historical_data('1y')
AAPL.data

/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.today_technical_indicators = pd.Series() # All techincal indicators for today
/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:18: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.labels = pd.Series() # Increased or decreased, compare to the stock price from 10 days ago


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-29,146.100006,146.720001,140.679993,142.479996,141.639725,128138200
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.384979,124925300
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.609909,114311700
2022-10-04,145.029999,146.220001,144.259995,146.100006,145.238403,87830100
2022-10-05,144.070007,147.380005,143.009995,146.399994,145.536606,79471000
...,...,...,...,...,...,...
2023-09-22,174.669998,177.080002,174.050003,174.789993,174.789993,56663000
2023-09-25,174.199997,176.970001,174.149994,176.080002,176.080002,46172700
2023-09-26,174.820007,175.199997,171.660004,171.960007,171.960007,64588900


In [5]:
AAPL.get_technical_indicators()
print(AAPL.data.shape)
AAPL.data.head()

(251, 18)


,Open,High,Low,Close,Adj Close,Volume,SMA_20,SMA_50,SMA_200,RSI,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,stochastic_k,stochastic_d
Date,,,,,,,,,,,,,,,,,,
2022-09-29,146.100006,146.720001,140.679993,142.479996,141.639725,128138200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.384979,124925300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.609909,114311700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-04,145.029999,146.220001,144.259995,146.100006,145.238403,87830100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-05,144.070007,147.380005,143.009995,146.399994,145.536606,79471000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(AAPL.technical_indicators.shape)
AAPL.technical_indicators.head()

(42, 12)


,SMA_20,SMA_50,SMA_200,upper_band,middle_band,lower_band,RSI,macd,macd_signal,macd_hist,stochastic_k,stochastic_d
Date,,,,,,,,,,,,
2023-07-18,189.524000,181.959401,156.68730,195.309010,189.524000,183.738990,67.782561,3.038319,3.223175,-0.184856,81.315856,69.962127
2023-07-19,190.028500,182.390001,156.95040,195.909944,190.028500,184.147057,70.121179,3.158424,3.210225,-0.051801,86.002459,79.312801
2023-07-20,190.487000,182.782601,157.22505,195.807623,190.487000,185.166377,63.035510,3.059379,3.180055,-0.120677,73.239021,80.185779
2023-07-21,190.734000,183.186001,157.47250,195.838450,190.734000,185.629551,59.147444,2.851986,3.114442,-0.262456,58.383529,72.541670
2023-07-24,191.037501,183.569801,157.70575,195.855468,191.037501,186.219533,60.914633,2.721612,3.035876,-0.314263,51.648051,61.090201


In [7]:
AAPL.today_technical_indicators

SMA_20          177.835500
SMA_50          181.704200
SMA_200         166.088450
upper_band      188.860987
middle_band     177.835500
lower_band      166.810014
RSI              37.007197
macd             -2.714773
macd_signal      -2.142099
macd_hist        -0.572673
stochastic_k     13.316744
stochastic_d     11.105082
Name: 2023-09-28 00:00:00, dtype: float64

# Stock Screener

### Run example with 3 selected stocks

In [8]:
# Define the required filters here for each technical indicator
filters = [lambda stock: filter_price(stock, 60, 200),
           lambda stock: filter_technical_indicator(stock, 'SMA_20', '>', 120)]

# Create list of Stock instance
test_stocks = [Stock('AAPL'), Stock('GOOG'), Stock('AAIC')]

# Initialize the screener with stocks and filters
screener = StockScreener(test_stocks, filters)

# Add 1 year historical data and technical indicators to stocks
screener.add_data()
filtered_stocks = screener.apply_filters()
filtered_stocks_ticker = [stock.ticker for stock in filtered_stocks]
filtered_stocks_ticker

/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.today_technical_indicators = pd.Series() # All techincal indicators for today
/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:18: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.labels = pd.Series() # Increased or decreased, compare to the stock price from 10 days ago
/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.today_technical_indicators = pd.Series() # All techincal indicators for today
/Users/ha

['AAPL', 'GOOG']

In [9]:
# Train model
screener.train_models()

# Make predictions and keep only those indicating an increase on the 10th day.
predicted_stocks = screener.predict_stocks(filtered_stocks)
print(len(predicted_stocks))

for stock in filtered_stocks:
    print(stock.prediction, stock.ticker)

2023-09-29 12:36:12.022657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 75ms/step
1
[[0.22815491]] AAPL
[[0.9559398]] GOOG


### Run example with stocks under 'real_estate' topics

In [10]:
# Define the required filters here for each technical indicator
filters = [lambda stock: filter_price(stock, 60, 200),
           lambda stock: filter_technical_indicator(stock, 'SMA_20', '>', 120)]

# Create list of Stock instance
real_estate_stocks = []
for ticker in all_stocks[all_stocks['Topics'] == 'real_estate']['Symbol']:
    real_estate_stocks.append(Stock(ticker))

# Initialize the screener with stocks and filters
screener = StockScreener(real_estate_stocks, filters)

# Add 1 year historical data and technical indicators to stocks
screener.add_data()

# Filter 1: filter stocks based on historical price and technical indicators
filtered_stocks = screener.apply_filters()
filtered_stocks_ticker = [stock.ticker for stock in filtered_stocks]
filtered_stocks_ticker

/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.today_technical_indicators = pd.Series() # All techincal indicators for today
/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:18: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.labels = pd.Series() # Increased or decreased, compare to the stock price from 10 days ago
/Users/haoyuwen/code/kychee2013/Stock_Portfolio_Prediction/modules/stock.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.today_technical_indicators = pd.Series() # All techincal indicators for today
/Users/ha

['ALX', 'AMT', 'AVB', 'DLR', 'EGP', 'EXR', 'MAA', 'SUI', 'SBAC']

In [11]:
# Train model
screener.train_models()

# Make predictions and keep only those indicating an increase on the 10th day.
predicted_stocks = screener.predict_stocks(filtered_stocks)
print(len(predicted_stocks), [x.ticker for x in predicted_stocks])

for stock in filtered_stocks:
    print(stock.prediction, stock.ticker)

1/1 [==============================] - 0s 60ms/step
4 ['ALX', 'AMT', 'DLR', 'EGP']
[[0.98247117]] ALX
[[0.5967276]] AMT
[[0.31154886]] AVB
[[0.9984538]] DLR
[[0.83424383]] EGP
[[0.10784133]] EXR
[[0.02234059]] MAA
[[0.33204845]] SUI
[[0.11082394]] SBAC


# ML Model

In [12]:
# Creating the figure and add Line trace
fig = go.Figure()
for stock in predicted_stocks:
    stock.get_historical_data('5y')
    stock.get_technical_indicators()
    fig.add_trace(go.Scatter(x=stock.data.index, y=stock.data['Close'], mode='lines', name=stock.ticker))

# Set the Layout
fig.update_layout(
    title="Stock - Close Price History",
    xaxis=dict(title='Date', showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title='Close Price USD ($)',showgrid=True, gridcolor='lightgray'),
    legend=dict(x=0.01, y=0.99, bordercolor='black', borderwidth=1),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
)

# Show the figure
fig.show()

In [13]:
# Predict selected stock price(Close)
for stock in predicted_stocks:
    print(stock.ticker)
    
    # Create a new dataframe and drop all NaN values
    data = stock.data.dropna()

    # Convert the dataframe to a numpy array
    dataset = data.values

    # Scale the data    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset)

    # Split X and y
    X = []
    y = []

    for i in range(60, len(scaled_data)-20):
        X.append(scaled_data[i-60:i, 6:])
        y.append(scaled_data[i:i+21, 3])

    X, y = np.array(X), np.array(y)
    print(X.shape, y.shape)
    
    # Initialize and train the model
    model = initialize_model((X.shape[1], X.shape[2]))
    model = compile_model(model)
    model, history = train_model(model, 
                                X=X, 
                                y=y, 
                                batch_size=1, 
                                epochs=7, 
                                patience=5, 
                                validation_split=0.2)
    stock.model = model
    
    # Predict future 1 month close price
    pred = model.predict(scaled_data[-60:, 6:].reshape(-1, 60, 12))
    inverse_scale = np.vectorize(lambda x: x*scaler.data_range_[3] + scaler.data_min_[3])
    pred = inverse_scale(pred)
    
    stock.pred = pred.flatten()

ALX
(978, 60, 12) (978, 21)
✅ Model initialized
✅ Model compiled
✅ Model trained on 978 rows with min val MAE: 0.05
1/1 [==============================] - 1s 839ms/step
AMT
(978, 60, 12) (978, 21)
✅ Model initialized
✅ Model compiled
Restoring model weights from the end of the best epoch: 1.
Epoch 6: early stopping
✅ Model trained on 978 rows with min val MAE: 0.09
1/1 [==============================] - 1s 842ms/step
DLR
(978, 60, 12) (978, 21)
✅ Model initialized
✅ Model compiled
Restoring model weights from the end of the best epoch: 1.
Epoch 6: early stopping
✅ Model trained on 978 rows with min val MAE: 0.06
1/1 [==============================] - 1s 896ms/step
EGP
(978, 60, 12) (978, 21)
✅ Model initialized
✅ Model compiled
✅ Model trained on 978 rows with min val MAE: 0.04
1/1 [==============================] - 1s 1s/step


In [14]:
# Convert predicted 1 month Close price to DataFrame
predicted_1mo = {}

for stock in predicted_stocks:
    predicted_1mo[stock.ticker] = stock.pred

pd.DataFrame.from_dict(predicted_1mo)

,ALX,AMT,DLR,EGP
0,189.999125,182.487559,119.820674,163.612070
1,189.139746,180.818185,119.089983,166.269999
2,196.969727,185.747962,118.108680,165.759710
3,198.141616,183.667509,123.067551,169.495883
4,192.833005,180.837940,121.108410,165.096067
5,197.002948,184.201223,123.012259,167.238652
6,198.660055,189.763668,119.969791,166.170812
7,195.050301,188.800798,120.563088,167.169090
8,193.593204,186.103889,120.868739,168.588589
9,195.028618,183.375421,123.340145,169.499952


In [15]:
predicted_1mo_df = pd.DataFrame.from_dict(predicted_1mo)

weights_max_sharpe(predicted_1mo_df, 21)

Expected annual return: 8.5%
Annual volatility: 4.5%
Sharpe Ratio: 1.45


OrderedDict([('ALX', -0.28229),
             ('AMT', 0.42018),
             ('DLR', 0.1266),
             ('EGP', 0.7355)])